In [1]:
import requests
import pandas as pd
import toml
import os
import sqlite3
import uuid

In [2]:
def get_github_folders(url):
  """
  Retrieves a list of folder names from a GitHub repository URL.

  Args:
    url: The URL of the GitHub repository.

  Returns:
    A list of folder names.
  """
  response = requests.get(url, headers={"Authorization": f"Bearer {os.getenv('go_blockchain_ecosystem')}"})
  response.raise_for_status()  # Raise an exception for bad status codes

  folders = []
  for item in response.json():
    if item['type'] == 'dir':
      folders.append(item['name'])
  return folders

In [3]:
def get_toml_files(url):
  """
  Retrieves a list of .toml files from a GitHub repository URL.

  Args:
    url: The URL of the GitHub repository.

  Returns:
    A list of .toml file names.
  """
  response = requests.get(url, headers={"Authorization": f"Bearer {os.getenv('go_blockchain_ecosystem')}"})
  response.raise_for_status()

  toml_files = []
  for item in response.json():
    if item['type'] == 'file' and item['name'].endswith('.toml'):
      toml_files.append(item['name'])
  return toml_files

In [4]:
def get_toml_data(url):
  """
  Retrieves and parses data from a .toml file at a given URL.

  Args:
    url: The URL of the .toml file.

  Returns:
    A dictionary containing the parsed data.
  """
  response = requests.get(url, headers={"Authorization": f"Bearer {os.getenv('go_blockchain_ecosystem')}"})
  response.raise_for_status()
  return toml.loads(response.text)

In [5]:
def main():
  """
  Main function to collect and process data from the GitHub repository.
  """
  base_url = "https://api.github.com/repos/electric-capital/crypto-ecosystems/contents/data/ecosystems"
  base_url_raw = "https://raw.githubusercontent.com/electric-capital/crypto-ecosystems/master/data/ecosystems"
  folders = get_github_folders(base_url)

  all_data = []
  for folder in folders:
    # if folder == '1':  # Check if it's folder '8'
    #     break  # Exit the loop if it is
    folder_url = f"{base_url}/{folder}"
    toml_files = get_toml_files(folder_url)

    for toml_file in toml_files:
      file_url = f"{base_url_raw}/{folder}/{toml_file}"
      try:
        data = get_toml_data(file_url)
        data['folder'] = folder  # Add folder information
        data['file'] = toml_file  # Add file name
        all_data.append(data)
      except toml.TomlDecodeError as e:
        print(f"Error decoding TOML file {file_url}: {e}")
        
  return pd.DataFrame(all_data)
  # You can further process or save the dataframe as needed

In [6]:
df = main()

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7190 entries, 0 to 7189
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   title                 7190 non-null   object
 1   sub_ecosystems        7190 non-null   object
 2   github_organizations  7190 non-null   object
 3   repo                  7129 non-null   object
 4   folder                7190 non-null   object
 5   file                  7190 non-null   object
dtypes: object(6)
memory usage: 337.2+ KB


In [8]:
df.head()

,title,sub_ecosystems,github_organizations,repo,folder,file
0,0.exchange,[],[https://github.com/zeroexchange],[{'url': 'https://github.com/zeroexchange/0-ch...,0,0-exchange.toml
1,01coin,[],[https://github.com/zocteam],[{'url': 'https://github.com/zocteam/bisq-asse...,0,01coin.toml
2,0cash,[],[],[{'url': 'https://github.com/0cash/0cash'}],0,0cash.toml
3,0chain,[],[https://github.com/0chain],"[{'url': 'https://github.com/0chain/0block', '...",0,0chain.toml
4,0g Labs,[],[https://github.com/0glabs],[{'url': 'https://github.com/0glabs/0g-avs-ser...,0,0g-labs.toml


In [9]:
# write df to sqlite database locally
# # first create a lookup column that assigns a unique id to each row
df['id'] = [str(uuid.uuid4()) for _ in range(len(df))]

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7190 entries, 0 to 7189
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   title                 7190 non-null   object
 1   sub_ecosystems        7190 non-null   object
 2   github_organizations  7190 non-null   object
 3   repo                  7129 non-null   object
 4   folder                7190 non-null   object
 5   file                  7190 non-null   object
 6   id                    7190 non-null   object
dtypes: object(7)
memory usage: 393.3+ KB


In [11]:
df.head()

,title,sub_ecosystems,github_organizations,repo,folder,file,id
0,0.exchange,[],[https://github.com/zeroexchange],[{'url': 'https://github.com/zeroexchange/0-ch...,0,0-exchange.toml,39289efd-6b8a-4757-b71f-587b356bcf0a
1,01coin,[],[https://github.com/zocteam],[{'url': 'https://github.com/zocteam/bisq-asse...,0,01coin.toml,76e9c2c1-76fa-4315-b3e7-8eac6958c19e
2,0cash,[],[],[{'url': 'https://github.com/0cash/0cash'}],0,0cash.toml,8027f0fb-6f78-4312-8a06-81e2b5b59ba1
3,0chain,[],[https://github.com/0chain],"[{'url': 'https://github.com/0chain/0block', '...",0,0chain.toml,54fc756c-ac61-4fc0-86a2-6eda1bdf3bab
4,0g Labs,[],[https://github.com/0glabs],[{'url': 'https://github.com/0glabs/0g-avs-ser...,0,0g-labs.toml,3d9a2074-8416-4be4-97e6-753c239de9ef


In [30]:
# next create a sqlite table that contains the following columns:
# id
# title
# file
# folder
# drop the pandas index
# manually define column names. Use unpacked_df, which will be a df used for later database ops
unpacked_df = df[['id', 'title', 'file', 'folder']].copy()
unpacked_df.columns = ['project_id', 'project_name', 'file', 'folder']
unpacked_df.to_sql('crypto_ecosystems', con=sqlite3.connect('crypto_ecosystems.db'), if_exists='replace', index=False)

In [31]:
# next, unpack the fields sub_ecosystems, github_organizations, repo into separate tables with the primary key being the id column
# and the foreign key being the id column in the crypto_ecosystems table

# the field sub_ecosystems contains a list of comma separated values
# the field github_organizations contains a list of comma separated values
# the field repo contain lists of dictionaries with the following key: url

# Function to unpack lists and create new rows
def unpack_list(row, column):
    org_id = row['id']
    title = row['title']
    github_orgs = row[column]
    data = []
    for org in github_orgs:
        data.append((org_id, title, org))  # Create a tuple for each value
    return data

In [32]:
# github_organizations
new_rows = df[['id', 'title', 'github_organizations']].apply(lambda row: unpack_list(row, 'github_organizations'), axis=1).explode()

# Create a new DataFrame from the unpacked data
unpacked_df = pd.DataFrame(new_rows.tolist(), columns=['id', 'title', 'github_organization'])

# drop rows where id is NaN
unpacked_df = unpacked_df[unpacked_df['id'].notna()]

# write unpacked_df to sqlite database locally
# manually define column names
unpacked_df.columns = ['project_id', 'project_name', 'github_org_url']
unpacked_df.to_sql('github_organizations', con=sqlite3.connect('crypto_ecosystems.db'), if_exists='replace', index=False)

In [33]:
# sub_ecosystems
# remove df rows that contain empty sub ecosystem lists
df_ecosystems = df[df['sub_ecosystems'].apply(lambda x: len(x) > 0)]

# Apply the function to each row and explode the resulting list
new_rows = df_ecosystems[['id', 'title', 'sub_ecosystems']].apply(lambda row: unpack_list(row, 'sub_ecosystems'), axis=1).explode()

# Create a new DataFrame from the unpacked data
unpacked_df = pd.DataFrame(new_rows.tolist(), columns=['id', 'title', 'sub_ecosystems'])

# drop rows where id is NaN
unpacked_df = unpacked_df[unpacked_df['id'].notna()]

# write unpacked_df to sqlite database locally
# manually define column names
unpacked_df.columns = ['project_id', 'project_name', 'sub_ecosystems']
unpacked_df.to_sql('sub_ecosystems', con=sqlite3.connect('crypto_ecosystems.db'), if_exists='replace', index=False)

In [34]:
# repos

# Function to unpack lists of dictionaries and create new rows
def unpack_list_of_dicts(row):
    org_id = row['id']
    title = row['title']
    org_repos = row['repo']
    data = []
    if isinstance(org_repos, list):  # Check if org_repos is a list
        for org in org_repos:
            if isinstance(org, dict) and 'url' in org:  # Check if org is a dictionary with 'url' key
                data.append((org_id, title, org['url']))
    return data

# Apply the function to each row and explode the resulting list
new_rows = df.apply(unpack_list_of_dicts, axis=1).explode()

# Create a new DataFrame from the unpacked data
unpacked_df = pd.DataFrame(new_rows.tolist(), columns=['id','title', 'url'])  # Use 'url' as column name

# write unpacked_df to sqlite database locally
# manually define column names
unpacked_df.columns = ['project_id', 'project_name', 'repo_url']
unpacked_df.to_sql('repos', con=sqlite3.connect('crypto_ecosystems.db'), if_exists='replace', index=False)